# Collaborative Filtering from Scratch Abbreviated

This notebook is used to demonstrate how to code and evaluate CF (Collaborative Filtering) using `sklearn`, `pandas`, `scipy`.

It is called "from scratch" as it does not use the `surprise` library.

## 1. Load libraries and data, inspect data, convert data as sparse

In [4]:
import os
import pickle
import time

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy as sp
from sklearn.model_selection import KFold, train_test_split
from sklearn.neighbors import NearestNeighbors

Specify your `ratings.csv`'s directory. The csv has 10k books, 50k users, 6M books, with ratings given between 1-5.

In [5]:
%%time
csv_dir = "/home/zebalgebra/School/DVA/The-Last-Book-Bender/Data/Raw/" # edit this line
ratings_all_df = pd.read_csv(
    os.path.join(csv_dir, "ratings.csv")
)
ratings_all_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5976479 entries, 0 to 5976478
Data columns (total 3 columns):
 #   Column   Dtype
---  ------   -----
 0   user_id  int64
 1   book_id  int64
 2   rating   int64
dtypes: int64(3)
memory usage: 136.8 MB
CPU times: user 572 ms, sys: 63.8 ms, total: 635 ms
Wall time: 721 ms


In [6]:
%%time
ratings_all_csc = sp.sparse.csc_matrix(
    (
        np.array(ratings_all_df["rating"]),
        (
            np.array(ratings_all_df["book_id"]),
            np.array(ratings_all_df["user_id"])
        )
    )
)

CPU times: user 232 ms, sys: 44.7 ms, total: 276 ms
Wall time: 281 ms


You can save this as an `npz` object using `save_npz`, and load it using `load_npz` (which is extremely fast).

## 2. Doing train test split for CV (on users)

The following code block shows how to set up a 10-fold CV iterator.

In [7]:
%%time
kf = KFold(n_splits=10, shuffle=True, random_state=6242) # change this for other CV options
user_ids = np.arange(53424 + 1) # the column range of our sparse matrix is 0-53424
for fold, (user_ids_train, user_ids_test) in enumerate(kf.split(user_ids)):
    print(f"Splitting for fold {fold} with train size {len(user_ids_train)}, test size {len(user_ids_test)}")
    true_size = (10000 + 1, 53424 + 1)
    ratings_train_csc = ratings_all_csc[:, user_ids_train]
    ratings_test_csc = ratings_all_csc[:, user_ids_test]

Splitting for fold 0 with train size 48082, test size 5343
Splitting for fold 1 with train size 48082, test size 5343
Splitting for fold 2 with train size 48082, test size 5343
Splitting for fold 3 with train size 48082, test size 5343
Splitting for fold 4 with train size 48082, test size 5343
Splitting for fold 5 with train size 48083, test size 5342
Splitting for fold 6 with train size 48083, test size 5342
Splitting for fold 7 with train size 48083, test size 5342
Splitting for fold 8 with train size 48083, test size 5342
Splitting for fold 9 with train size 48083, test size 5342
CPU times: user 108 ms, sys: 78.6 ms, total: 186 ms
Wall time: 187 ms


## 3. Utilities for Shifting Ratings

The following code block defines utility functions for modifying underlying data of a csc matrix.

In [8]:
def dilate(X: sp.sparse.csc_matrix, amount: float) -> None:
    """
    deduct each nonempty entry of X by amount
    (this mutates the underlying csc matrix)
    for example, if take amount as baseline,
    this function is useful.
    """
    X.data = X.data - np.float64(amount)

def value_remap(X: sp.sparse.csc_matrix, remapper: np.ndarray) -> None:
    """
    remap nonnull values of X based on remapper
    (if remapper[i]=j, value of i is remapped to j)
    note this function takes an np array (not an ordinary list)
    """
    X.data = remapper[X.data.astype(int)].astype(np.float64)

def make_mean_0(X: sp.sparse.csc_matrix) -> None:
    """
    make columns of a csc_matrix have zero mean
    (this function mutates the underlying data)
    """
    X.data -= get_true_mean(X)

def get_true_mean(X: sp.sparse.csc_matrix) -> None:
    """
    compute mean of each column (over nonzero indices!)
    this won't mutate the underlying X
    a general helper function
    """
    indexer = X.tocoo().col
    v = np.array(X.sum(axis=0)).flatten()
    c = np.array(X.minimum(1).sum(axis=0)).flatten()
    return v[indexer] / np.maximum(c[indexer], 0.5)

def test_mean_is_0(X: sp.sparse.csc_matrix, tol=10 ** (-10)) -> np.bool_:
    """
    test if mean is 0 for each column
    should be true on outputs of make_mean_0
    """
    return np.abs(np.array(X.sum(axis=0))).max() < tol

In [9]:
%%time
ratings_train_csc_demo = ratings_train_csc.copy().astype(np.float64)
value_remap(ratings_train_csc_demo, np.array([0, -1, -0.5, 0.25, 0.5, 1]))
# dilate(ratings_train_csc_demo, 2.5)
# make_mean_0(ratings_train_csc_demo)
# you can test if make_mean_0 is correct by uncommenting the next line
# test_mean_is_0(ratings_train_csc_demo)
print(np.vstack([ratings_train_csc.data, ratings_train_csc_demo.data])[:, :15])

[[ 5.    4.    5.    4.    3.    3.    4.    4.    4.    5.    4.    2.
   2.    3.    4.  ]
 [ 1.    0.5   1.    0.5   0.25  0.25  0.5   0.5   0.5   1.    0.5  -0.5
  -0.5   0.25  0.5 ]]
CPU times: user 30.6 ms, sys: 43.7 ms, total: 74.3 ms
Wall time: 74.1 ms


## 4. Getting Top Similarity Scores and Neighbors

With these functions, we can plug these into `NearestNeighbors` to get top recommendations based on our desired metrics. Take adjusted cosine as an example.

In [10]:
%%time
# preprocess train and test sets
ratings_train_csc_modified = ratings_train_csc.copy().astype(np.float64)
ratings_test_csc_modified = ratings_test_csc.copy().astype(np.float64)
# value_remap(ratings_train_csc_modified, np.array([0, -1, -0.5, 0.25, 0.5, 1]))
# value_remap(ratings_test_csc_modified, np.array([0, -1, -0.5, 0.25, 0.5, 1]))
make_mean_0(ratings_train_csc_modified)
make_mean_0(ratings_test_csc_modified)
# fit knn
n_neighbors = 5
knn = NearestNeighbors(metric="cosine")
knn.fit(ratings_train_csc_modified.transpose()) # sklearn's convention is to use rows
neigh_dist, neigh_ind = knn.kneighbors(
    X=ratings_test_csc_modified.transpose(),
    n_neighbors=n_neighbors
) # this step took ~12 seconds on my machine
sim_scores = 1 - neigh_dist
sim_scores.shape, neigh_ind.shape

CPU times: user 10.9 s, sys: 1.12 s, total: 12 s
Wall time: 12.1 s


((5342, 5), (5342, 5))

We abstract this procedure.

In [11]:
def get_top_neigh_dist_ind(
    ratings_train_csc_modified,
    ratings_test_csc_modified,
    n_neighbors=5
):
    knn = NearestNeighbors(metric="cosine")
    knn.fit(ratings_train_csc_modified.transpose())
    neigh_dist, neigh_ind = knn.kneighbors(
        X=ratings_test_csc_modified.transpose(),
        n_neighbors=n_neighbors
    )
    return 1 - neigh_dist, neigh_ind

## 5. Predict Ratings

Now we have rankings and similarity scores. To evaluate model performance - such as MAP, MSE - we need to predict ratings on test user's ratings. We only evaluate on those with ground truths.

Fix a number $k$ for desired neighborhood size. We use the approximated metric (here $u$ is a user, $i$ is an item):

$$\hat{r}_{ui}=\bar{r}_u+\frac{\sum_{v\in N_u}\delta_{i, I_v}\text{sim}_{uv}(r_{vi}-\bar{r}_v)}{\sum_{v\in N_u}\delta_{i, I_v}|\text{sim}_{uv}|},\quad\text{for test user }u,\text{ an item }i\text{ that }u\text{ has rated}$$

where $N_{u}$ is the set of top $k$ neighbors of $u$, $\text{sim}_{uv}$ is similarity score between $u,v$, $r_{vi}$ is true rating. Although this metric is a suboptimal one, this allows more vectorization to our computation.

In [12]:
%%time
# compute vector of means
ratings_train_csc_mean = get_true_mean(ratings_train_csc)
ratings_test_csc_mean = get_true_mean(ratings_test_csc)

CPU times: user 108 ms, sys: 20.4 ms, total: 128 ms
Wall time: 128 ms


In [13]:
%%time
# get number of total items, test users
n_items = ratings_test_csc.shape[0]
n_users_test = len(user_ids_test)

# initialize arrays
ratings_test_csc_predicted = ratings_test_csc.copy()
ratings_test_csc_predicted.data = np.zeros(len(ratings_test_csc.data))

# compute vector of means
ratings_train_mean = get_true_mean(ratings_train_csc)
ratings_test_mean = get_true_mean(ratings_test_csc)

# these two sprase arrays will represent the numerator, denominator involving the delta terms
numer_test_csc_ratings = ratings_test_csc_predicted.copy()
denom_test_csc_ratings = ratings_test_csc_predicted.copy()

# prediction loop
start_ind = 0
for i in np.arange(n_users_test):
    denom_col = np.zeros(n_items)
    numer_col = np.zeros(n_items)
    baseline_items = ratings_test_csc.indices[
        ratings_test_csc.indptr[i]: ratings_test_csc.indptr[i + 1]
    ]
    n_baseline_items = len(baseline_items)
    end_ind = start_ind + n_baseline_items
    for j, sim_score in zip(neigh_ind[i], sim_scores[i]):
        r_v = ratings_train_csc[:, j].toarray().flatten()[baseline_items]
        mu_v = ratings_train_mean[j]
        w = sim_score * np.minimum(1, r_v)
        numer_test_csc_ratings.data[start_ind: end_ind] += w * (r_v - mu_v)
        denom_test_csc_ratings.data[start_ind: end_ind] += w
    start_ind = end_ind
ratings_test_csc_predicted.data =\
    ratings_test_mean[ratings_test_csc_predicted.tocoo().col] +\
    numer_test_csc_ratings.data /\
    np.maximum(
        denom_test_csc_ratings.data,
        10 ** (-30)
    )
ratings_diffs = ratings_test_csc.data - ratings_test_csc_predicted.data

CPU times: user 3.09 s, sys: 33.1 ms, total: 3.13 s
Wall time: 3.13 s


For future reference, we abstract the rating prediction part as a function. Since we will test different neighborhood sizes, to avoid repeated calculation, some modifications are made.

In [14]:
def update_numer_denom(
    numer_test_csc_ratings,
    denom_test_csc_ratings,
    ratings_test_csc,
    ratings_train_csc,
    ratings_train_mean,
    n_items,
    n_users_test,
    pos # position of which neighbor to update
):
    """
    this function mutates the input csc matrices
    """
    start_ind = 0
    for i in np.arange(n_users_test):
        denom_col = np.zeros(n_items)
        numer_col = np.zeros(n_items)
        baseline_items = ratings_test_csc.indices[
            ratings_test_csc.indptr[i]: ratings_test_csc.indptr[i + 1]
        ]
        n_baseline_items = len(baseline_items)
        end_ind = start_ind + n_baseline_items
        j = neigh_ind[i, pos]
        sim_score = sim_scores[i, pos]
        r_v = ratings_train_csc[:, j]\
                .toarray().flatten()[baseline_items]
        mu_v = ratings_train_mean[j]
        w = sim_score * np.minimum(1, r_v)
        numer_test_csc_ratings.data[start_ind: end_ind] += w * (r_v - mu_v)
        denom_test_csc_ratings.data[start_ind: end_ind] += w
        start_ind = end_ind

In [15]:
def update_prediction(
    ratings_test_csc_predicted,
    ratings_test_mean,
    numer_test_csc_ratings,
    denom_test_csc_ratings
):
    ratings_test_csc_predicted.data =\
        ratings_test_mean[ratings_test_csc_predicted.tocoo().col] +\
        numer_test_csc_ratings.data /\
        np.maximum(
            denom_test_csc_ratings.data,
            10 ** (-30)
        )

## 3.4 Collaborative Filtering - Evaluating ratings

Let us evaluate our model using just RMSE, MAE - in an offline setting and with only rating values between 1, 2, 3, 4, 5, RMSE and MAE are the most straightforward ones.

The formulas:

$$\text{RMSE}=\sqrt{|T|^{-1}\sum_{(u, i)\in T}(r_{ui}-\hat{r}_{ui})^2}$$

$$\text{MAE}={|T|^{-1}\sum_{(u, i)\in T}|r_{ui}-\hat{r}_{ui}|}$$

More succinctly, RMSE, MAE corresponds to the 2 and 1 - norm of the difference between predictions and ratings. We can compute them using `numpy.linalg.norm`:

In [16]:
def eval_error(ratings_diffs: np.ndarray, sense:str="RMSE"):
    """
    function for evaluating RMSE, MAE of a ratings_diffs array
    sense can be "RMSE" or "MAE", no other options for now
    """
    if sense not in {"RMSE", "MAE"}:
        raise NotImplementedError
    p = {"RMSE": 2, "MAE": 1}[sense]
    return np.linalg.norm(ratings_diffs, p) / ratings_diffs.shape[0] ** (1 / p)
print(f'RMSE (root mean square error): {eval_error(ratings_diffs, "RMSE")}')
print(f'MAE (mean absolute error): {eval_error(ratings_diffs, "MAE")}')

RMSE (root mean square error): 1.0384170135083597
MAE (mean absolute error): 0.8331371945569175


We can probably do better than this.

## 3.5 Collaborative Filtering - Putting all together

Let us make clear on which models that we want to try:
1. Number of folds for CV - we fix this to be 10 for our task.
2. Metrics to use - we use shifting by 0 (ordinary cosine), 2.5, 2.75, 2.9, 3, adjusted cosine, and the mapping of 1-2 to -1, 3-5 to 1.
3. Number of neighborhoods to use - we use 3, 5, 10, 12, 15, 20, 30 (note that the time neede to generate predictions is roughly linear to number of neighborhoods used)

The evaluation pipeline:
* (A) Load data and convert to sparse matrix
* (B) For each train test split index for a 10 fold CV:
    * (B1) Build train test set (by sparse array column slicing), Calculate some metadata
    * (B2) For every metrics that we want to use:
        * (B2-1) Copy train test set and modify its values
        * (B2-2) Build knn model from train set, and fit this model to test set to grab 200 nearest neighbors (will time this step) and simimilarity scores.
        * (B2-3) For for each number of neighborhoods we want to try:
            * (B2-3-1) Update the numerator, denominator term, update predicted scores for each user-item in train set (will time this step).
            * (B2-3-2) Get difference of ratings (predicted versus baseline in train set)
            * (B2-3-3) Calculate MAE and RMSE.
            * (B2-3-4) Save MAE, RMSE, prediction generation time.
* (C) Combine (MAE, RMSE, prediction generation time) for each pair of (metric, number of neighborhood) across the 10 folds.

In [20]:
%%time
results = []
# specify metadata
metric_names = [
                "cosine", *[f"deduct {amount}" for amount in [2.5, 2.75, 2.9, 3]],
                "adjusted cosine",
                "remap 12->-1, 345->1",
                "remap 1,5->-+1, 24->-+0.5, 3->0.25"
            ]
metric_funcs = [
                lambda x: dilate(x, 0),
                lambda x: dilate(x, 2.5),
                lambda x: dilate(x, 2.75),
                lambda x: dilate(x, 2.9),
                lambda x: dilate(x, 3),
                make_mean_0,
                lambda x: value_remap(x, np.array([0, -1, -1, 1, 1, 1])),
                lambda x: value_remap(x, np.array([0, -1, -0.5, 0.25, 0.5, 1]))
            ]
n_neighbors = 200
# (A)
csv_dir = "/home/zebalgebra/School/DVA/The-Last-Book-Bender/Data/Raw/"
ratings_all_df = pd.read_csv(
    os.path.join(csv_dir, "ratings.csv")
)
ratings_all_csc = sp.sparse.csc_matrix(
    (
        ratings_all_df["rating"],
        (
            ratings_all_df["book_id"],
            ratings_all_df["user_id"]
        )
    )
)
# (B)
kf = KFold(n_splits=10, shuffle=True, random_state=6242)
user_ids = np.arange(53424 + 1)
for (fold, (user_ids_train, user_ids_test)) in enumerate(kf.split(user_ids)):
    print(f"fold={fold} started.")
    fold_time_start = time.time()
    # (B1)
    ratings_train_csc = ratings_all_csc[:, user_ids_train]
    ratings_test_csc = ratings_all_csc[:, user_ids_test]
    ratings_train_mean = get_true_mean(ratings_train_csc)
    ratings_test_mean = get_true_mean(ratings_test_csc)
    n_items = ratings_test_csc.shape[0]
    n_users_test = len(user_ids_test)
    # (B2)
    for metric_name, metric_func in zip(metric_names, metric_funcs):
        print(f" - fold={fold}, metric='{metric_name}' started.")
        # (B2-1)
        ratings_train_csc_modified = ratings_train_csc.copy().astype(np.float64)
        ratings_test_csc_modified = ratings_test_csc.copy().astype(np.float64)
        metric_func(ratings_train_csc_modified)
        metric_func(ratings_test_csc_modified)
        # (B2-2)
        sim_scores, neigh_ind = get_top_neigh_dist_ind(
            ratings_train_csc_modified,
            ratings_test_csc_modified,
            n_neighbors=n_neighbors
        )
        # (B2-3)
        ratings_test_csc_predicted = ratings_test_csc.copy()
        ratings_test_csc_predicted.data = np.zeros(len(ratings_test_csc.data))
        numer_test_csc_ratings = ratings_test_csc_predicted.copy()
        denom_test_csc_ratings = ratings_test_csc_predicted.copy()
        for pos in range(n_neighbors):
            update_numer_denom(
                numer_test_csc_ratings,
                denom_test_csc_ratings,
                ratings_test_csc,
                ratings_train_csc,
                ratings_train_mean,
                n_items,
                n_users_test,
                pos
            )
            update_prediction(
                ratings_test_csc_predicted,
                ratings_test_mean,
                numer_test_csc_ratings,
                denom_test_csc_ratings
            )
            ratings_diffs = ratings_test_csc_predicted.data - ratings_test_csc.data
            # (B2-3-3)
            rmse = eval_error(ratings_diffs, "RMSE")
            mae = eval_error(ratings_diffs, "MAE")
            # (B2-3-4)
            results.append(
                {
                    "fold": fold,
                    "metric_name": metric_name,
                    "n_neighbors": pos + 1,
                    "rmse": rmse,
                    "mae": mae
                }
            )
    print(f"fold={fold} ended. used {time.time()-fold_time_start} seconds.")
    with open(f'checkpoint_fold_{fold}.pkl', 'wb') as f:
        pickle.dump(results, f)

fold=0 started.
 - fold=0, metric='cosine' started.
 - fold=0, metric='deduct 2.5' started.
 - fold=0, metric='deduct 2.75' started.
 - fold=0, metric='deduct 2.9' started.
 - fold=0, metric='deduct 3' started.
 - fold=0, metric='adjusted cosine' started.
 - fold=0, metric='remap 12->-1, 345->1' started.
 - fold=0, metric='remap 1,5->-+1, 24->-+0.5, 3->0.25' started.
fold=0 ended. used 1316.2100937366486 seconds.
fold=1 started.
 - fold=1, metric='cosine' started.
 - fold=1, metric='deduct 2.5' started.
 - fold=1, metric='deduct 2.75' started.
 - fold=1, metric='deduct 2.9' started.
 - fold=1, metric='deduct 3' started.
 - fold=1, metric='adjusted cosine' started.
 - fold=1, metric='remap 12->-1, 345->1' started.
 - fold=1, metric='remap 1,5->-+1, 24->-+0.5, 3->0.25' started.
fold=1 ended. used 1329.7925810813904 seconds.
fold=2 started.
 - fold=2, metric='cosine' started.
 - fold=2, metric='deduct 2.5' started.
 - fold=2, metric='deduct 2.75' started.
 - fold=2, metric='deduct 2.9' st

The testing results are saved in `checkpoint_fold_0.pkl` - `checkpoint_fold_10.pkl`.

The plot for RMSE and MAE are in `CF-CV-Results.ipynb`.